In [0]:
import dataiku
from dataikuapi.utils import DataikuException

In [0]:
client = dataiku.api_client()

# Retrieve all plugins
plugins = client.list_plugins()

In [0]:
def get_installed_version_of_plugin(plugin_id):
    plugins = client.list_plugins()
    for plugin_info in plugins:
        if plugin_id == plugin_info['id']:
            return  plugin_info.get('version', 'unknown')
        
# get_installed_version_of_plugin('govern')

In [0]:
# Function to check and update plugins that have available updates
def update_plugins(plugins):
    for plugin_info in plugins:
        plugin_id = plugin_info['id']
        plugin = client.get_plugin(plugin_id)
      
        version_before = plugin_info.get('version', '0.0')

        try:
            # 5. Install updates when available
            print(f'Attempting to update plugin {plugin_id} ... ', end="")
            future = plugin.update_from_store()
            future.wait_for_result()

            version_after = get_installed_version_of_plugin(plugin_id)
            
            if version_before == version_after:
                print(f' No update available.')
            else:
                print(f" Updated plugin: {plugin_id} from {version_before} to version {version_after}")

#             # 6. Rebuild code environments if necessary
#             if 'bundle' in latest_version_info:
#                 print(f"Rebuilding code environment for plugin: {plugin_id}")
#                 env_future = plugin.rebuild_bundle_env()
#                 env_future.wait_for_result()
#                 print(f"Rebuilt code environment for plugin: {plugin_id}")
        except Exception as e:
            print(f"Failed to update {plugin_id}: {str(e)}")

update_plugins(plugins)

print("You may need to update the associated code environments for any plugins that were upgraded.")